In [1]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as L

from pytorch_lightning import Trainer
import wandb

import os

In [2]:
print(snn.__version__)
print(torch.__version__)

In [3]:
# Our data in in the shape: trainings samples(28k) * number of time steps (100 + 6) * time dimension(1)
# The time steps is voltage over time
data = torch.load(f'./data/dataset.ds')

print(data.keys())
print(data['X_train'].shape)
print(data['Y_train'].shape)

In [4]:
X_train = data['X_train'][1].squeeze().numpy()[:-6] # last 6 are parameters of the simulation i.e. not voltage

plt.plot(X_train)
plt.xlabel("Time step")
plt.ylabel("Voltage")
plt.grid(True)
plt.show()

In [5]:
# Extract tensors
X_train, Y_train = data['X_train'], data['Y_train']
X_valid, Y_valid = data['X_valid'], data['Y_valid']
X_test, Y_test = data['X_test'], data['Y_test']

train_dataset = TensorDataset(X_train, Y_train)
valid_dataset = TensorDataset(X_valid, Y_valid)
test_dataset  = TensorDataset(X_test, Y_test)

In [6]:
class SpikeSynth(L.LightningModule):
    def __init__(self, num_voltage_steps, num_params, num_inputs, num_hidden, num_outputs,
                 beta, optimizer_class, lr, train_dataset, valid_dataset, batch_size, gamma):
        super().__init__()
        
        # Save hyperparameters for easy access
        self.save_hyperparameters()

        # Network layers
        self.linear0 = nn.Linear(self.hparams.num_inputs, self.hparams.num_hidden)
        self.lif0 = snn.Leaky(beta=self.hparams.beta)
        self.linear1 = nn.Linear(self.hparams.num_hidden, self.hparams.num_hidden)
        self.lif1 = snn.Leaky(beta=self.hparams.beta)
        self.linear2 = nn.Linear(self.hparams.num_hidden, self.hparams.num_hidden)
        self.lif2 = snn.Leaky(beta=self.hparams.beta)
        self.linear3 = nn.Linear(self.hparams.num_hidden + self.hparams.num_params, self.hparams.num_outputs)

        # Datasets
        self.train_dataset = train_dataset
        self.valid_dataset = valid_dataset

    def forward(self, x):
        volt = x[:, :self.hparams.num_voltage_steps, :]
        params = x[:, self.hparams.num_voltage_steps:, 0]

        mem0 = self.lif0.init_leaky()
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        
        mem_rec = []
        for step in range(self.hparams.num_voltage_steps):
            x_step = self.linear0(volt[:, step, :])
            spk, mem0 = self.lif0(x_step, mem0)

            x_step = self.linear1(spk)
            spk, mem1 = self.lif1(x_step, mem1)
            
            x_step = self.linear2(spk)
            spk, mem2 = self.lif2(x_step, mem2)

            mem_rec.append(mem2)

        mem_final = torch.stack(mem_rec, dim=0)[-1]
        combined = torch.cat((mem_final, params), dim=1)
        out = self.linear3(combined)
        return out

    def training_step(self, batch, batch_idx):
        X_batch, y_batch = batch
        outputs = self(X_batch)
        loss = torch.nn.MSELoss()(outputs, y_batch.float())
        self.log("train_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        X_batch, y_batch = batch
        outputs = self(X_batch)
        loss = torch.nn.MSELoss()(outputs, y_batch.float())
        self.log("val_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def on_train_epoch_end(self):
        opt = self.optimizers()
        lr = opt.param_groups[0]["lr"]
        
        # Log the learning rate to W&B or progress bar
        self.log("lr", lr, prog_bar=True, on_step=False, on_epoch=True)


    def configure_optimizers(self):
        optimizer = self.hparams.optimizer_class(self.parameters(), lr=self.hparams.lr)
        
        #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        #    optimizer, mode='min'
        #)
        
        scheduler = torch.optim.lr_scheduler.ExponentialLR(
            optimizer, gamma=self.hparams.gamma
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "val_loss",
        }

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.hparams.batch_size,
            shuffle=True,
            num_workers=8,
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.valid_dataset,
            batch_size=self.hparams.batch_size,
            shuffle=False,
            num_workers=4,
        )

In [7]:
num_voltage_steps=100
num_params=6
num_inputs=1
num_hidden=10 
num_outputs=100
beta=0.9
lr=1e-3 # Since we use ReduceLROnPlateau we can start with a higher LR
gamma=0.95

batch_size = 2048

optimizer_class = torch.optim.Adam

experiment_name = "Debug"
epochs = 2

In [8]:
model = SpikeSynth(
    num_voltage_steps=num_voltage_steps,
    num_params=num_params,
    num_inputs=num_inputs,
    num_hidden=num_hidden,
    num_outputs=num_outputs,
    beta=beta,
    optimizer_class=optimizer_class,
    lr=lr,
    train_dataset=train_dataset,
    valid_dataset=valid_dataset,
    batch_size=batch_size,
    gamma=gamma
)

model = torch.compile(model)

In [9]:
# Create a CometLogger instance
wandb_logger = WandbLogger(
                          log_model=True,
                          project="Spike-Synth",
                          name=experiment_name,
                          )

# log gradients and model topology
wandb_logger.watch(model)
wandb_logger.experiment.log_code(".", include_fn=lambda path: path.endswith(".py") or path.endswith(".ipynb"))

<Artifact source-Spike-Synth-surrogate_Lighning_Pipeline.ipynb>

In [10]:
trainer = Trainer(
    # fast_dev_run=True,
    max_epochs=epochs,
    logger=wandb_logger,  
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
)

# Train the model
trainer.fit(model)

wandb_logger.experiment.finish()

Sanity Checking: |                            | 0/? [00:00<?, ?it/s]

Training: |                                   | 0/? [00:00<?, ?it/s]

Validation: |                                 | 0/? [00:00<?, ?it/s]

Validation: |                                 | 0/? [00:00<?, ?it/s]